In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import plotly.figure_factory as ff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

In [13]:
#import dataset
original_election_data = pd.read_csv("merged_train.csv")
original_election_data.head()

,State,County,FIPS,Total Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural,Democratic,Republican,Party
0,AZ,apache,4001,72346,18.571863,0.486551,5.947806,1.719515,50.598513,45.854643,13.322091,32460,15.807433,21.758252,88.941063,74.061076,16298,7810,1
1,AZ,cochise,4003,128177,56.299492,3.714395,34.403208,11.458374,49.069646,37.902276,19.756275,45383,8.567108,13.409171,76.837055,36.301067,17383,26929,0
2,AZ,coconino,4005,138064,54.619597,1.342855,13.711033,4.825298,50.581614,48.946141,10.873943,51106,8.238305,11.085381,65.791439,31.466066,34240,19249,1
3,AZ,gila,4007,53179,63.222325,0.552850,18.548675,4.249798,50.296170,32.238290,26.397638,40593,12.129932,15.729958,82.262624,41.062000,7643,12180,0
4,AZ,graham,4009,37529,51.461536,1.811932,32.097844,4.385942,46.313518,46.393456,12.315809,47422,14.424104,14.580797,86.675944,46.437399,3368,6870,0


In [14]:
#import dataset
demographic_data = pd.read_csv("demographics_test.csv")
demographic_data.head()

,State,County,FIPS,Total Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
0,NV,eureka,32011,1730,98.265896,0.057803,0.462428,0.346821,51.156069,27.109827,15.606936,70000,3.755365,8.415466,83.396513,100.000000
1,TX,zavala,48507,12107,5.798299,0.594697,93.326175,9.193029,49.723301,49.302057,12.480383,26639,11.955168,40.840797,90.869691,38.032029
2,VA,king george,51099,25260,73.804434,16.722090,4.441805,2.505938,50.166271,40.186065,11.868567,84342,6.479939,7.152824,65.540254,73.189450
3,OH,hamilton,39061,805965,66.354867,25.654340,2.890944,5.086945,51.870615,40.779686,14.161657,50399,7.864630,9.873275,64.404446,2.231877
4,TX,austin,48015,29107,63.809393,8.479060,25.502456,9.946061,50.671660,37.351840,17.799842,56681,5.782337,17.579456,79.008391,66.344090


In [16]:
demo_data = original_election_data[original_election_data['Party'] == 1]
repub_data = original_election_data[original_election_data['Party'] != 1]
demo_data.head()

,State,County,FIPS,Total Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural,Democratic,Republican,Party
0,AZ,apache,4001,72346,18.571863,0.486551,5.947806,1.719515,50.598513,45.854643,13.322091,32460,15.807433,21.758252,88.941063,74.061076,16298,7810,1
2,AZ,coconino,4005,138064,54.619597,1.342855,13.711033,4.825298,50.581614,48.946141,10.873943,51106,8.238305,11.085381,65.791439,31.466066,34240,19249,1
6,AZ,maricopa,4013,4088549,56.918114,5.013612,30.286833,14.729333,50.549278,41.886620,13.837843,55676,6.808454,13.051927,69.031137,2.363800,732671,672505,1
9,AZ,pima,4019,1003338,53.271579,3.199719,36.105978,12.903428,50.807405,40.087388,17.801778,46764,9.214114,12.252238,69.199391,7.523491,221242,160550,1
10,AZ,santa cruz,4023,46547,15.274883,0.199798,83.219112,32.644424,52.125808,43.300320,15.895761,38941,9.749896,25.206726,77.506775,26.883172,9241,3828,1


In [18]:
demo_x_train, demo_x_test, demo_y_train, demo_y_test = train_test_split(demo_data.loc[:, demo_data.columns != 'Democratic'], demo_data['Democratic'], test_size = 0.25, random_state=0)
demo_x_train.head()

,State,County,FIPS,Total Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural,Republican,Party
156,MD,anne arundel,24003,559737,70.303375,15.559093,7.039377,8.270491,50.540700,38.635109,13.405046,91918,5.531967,8.088706,60.634981,5.304135,92401,1
539,NY,tompkins,36109,104268,77.786090,3.584993,4.724364,12.787241,50.814248,49.566502,12.176315,54133,5.367129,5.495592,48.397747,43.343114,7300,1
914,TX,webb,48479,266006,3.482628,0.269543,95.479801,26.488876,51.283806,52.401450,8.584393,38711,5.934847,34.323451,82.546075,2.599639,13814,1
542,NY,washington,36115,62465,92.738333,2.859201,2.531017,2.214040,48.278236,34.244777,17.177619,51449,8.071506,11.474751,79.753985,67.898950,8596,1
479,NM,mora,35033,4598,13.853849,0.108743,79.904306,0.761201,48.368856,39.821662,20.748151,21190,2.925990,12.284197,86.819389,100.000000,372,1


In [19]:
repub_x_train, repub_x_test, repub_y_train, repub_y_test = train_test_split(repub_data.loc[:, repub_data.columns != 'Republican'], repub_data['Republican'], test_size = 0.25, random_state=0)
repub_x_train.head()

,State,County,FIPS,Total Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural,Democratic,Party
1062,WA,grant,53025,92530,55.510645,0.652761,40.121042,19.251054,49.605533,46.740517,12.576462,50145,9.068370,24.889474,84.383853,38.748878,8159,0
944,VA,amelia,51007,12793,72.203549,23.528492,1.188150,0.242320,48.080982,34.565778,18.166185,58269,4.030710,18.747199,85.925594,100.000000,1938,0
449,NE,thayer,31169,5163,95.680806,0.135580,2.208019,0.890955,50.455162,31.299632,25.353477,45590,1.439129,8.150722,80.411545,100.000000,539,0
742,TX,archer,48009,8750,88.582857,0.685714,8.251429,3.325714,50.354286,35.005714,17.794286,62407,3.334836,9.917898,78.193760,88.988292,376,0
398,NE,chase,31029,4061,84.806698,0.024624,13.568087,9.258803,51.169663,35.434622,20.906181,53496,1.542169,11.087268,81.509299,100.000000,184,0


In [20]:
scaler = StandardScaler()
scaler.fit(demo_x_train)
demo_x_train_scaled = scaler.transform(demo_x_train)
demo_x_test_scaled = scaler.transform(demo_x_test)

ValueError: could not convert string to float: 'MD'

In [ ]:
scaler = StandardScaler()
scaler.fit(repub_x_train)
repub_x_train_scaled = scaler.transform(repub_x_train)
repub_x_test_scaled = scaler.transform(repub_x_test)